In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv('SBAnational.csv')

In [ ]:
df_copy = df.copy()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['Name', 'City', 'State', 'BankState', 'NewExist','RevLineCr', 'LowDoc', 'DisbursementDate', 'MIS_Status'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']].head()

In [ ]:
df[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']] = \
df[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))

In [ ]:
df['ApprovalFY'].apply(type).value_counts()

In [ ]:
df.ApprovalFY.unique()

In [ ]:
def clean_str(x):
    if isinstance(x, str):
        return x.replace('A', '')
    return x

df.ApprovalFY = df.ApprovalFY.apply(clean_str).astype('int64')

In [ ]:
df['ApprovalFY'].apply(type).value_counts()

In [ ]:
df = df.astype({'Zip': 'str', 'NewExist': 'int64', 'UrbanRural': 'str', 'DisbursementGross': 'float', 'BalanceGross': 'float',
                          'ChgOffPrinGr': 'float', 'GrAppv': 'float', 'SBA_Appv': 'float'})

In [ ]:
df.dtypes

In [ ]:
df['Industry'] = df['NAICS'].astype('str').apply(lambda x: x[:2])

In [ ]:
df['Industry'] = df['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin'
})

In [ ]:
df.dropna(subset = ['Industry'], inplace = True)

In [ ]:
df.FranchiseCode.unique()

In [ ]:
df.loc[(df['FranchiseCode'] <= 1), 'IsFranchise'] = 0
df.loc[(df['FranchiseCode'] > 1), 'IsFranchise'] = 1

In [ ]:
df.FranchiseCode

In [ ]:
df = df[(df['NewExist'] == 1) | (df['NewExist'] == 2)]

df.loc[(df['NewExist'] == 1), 'NewBusiness'] = 0
df.loc[(df['NewExist'] == 2), 'NewBusiness'] = 1

In [ ]:
df.NewExist.unique()

In [ ]:
df.RevLineCr.unique()

In [ ]:
df.LowDoc.unique()

In [ ]:
df = df[(df.RevLineCr == 'Y') | (df.RevLineCr == 'N')]
df = df[(df.LowDoc == 'Y') | (df.LowDoc == 'N')]

df['RevLineCr'] = np.where(df['RevLineCr'] == 'N', 0, 1)
df['LowDoc'] = np.where(df['LowDoc'] == 'N', 0, 1)

In [ ]:
df.RevLineCr.unique()
df.LowDoc.unique()

In [ ]:
df.MIS_Status.unique()

In [ ]:
df.MIS_Status.value_counts()

In [ ]:
df['Default'] = np.where(df['MIS_Status'] == 'P I F', 0, 1)
df['Default'].value_counts()

In [ ]:
df[['ApprovalDate', 'DisbursementDate']] = df[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)

In [ ]:
df['DaysToDisbursement'] = df['DisbursementDate'] - df['ApprovalDate']

In [ ]:
df.DaysToDisbursement.info()

In [ ]:
df['DaysToDisbursement'] = df['DaysToDisbursement'].astype('str').apply(lambda x: x[:x.index('d') - 1]).astype('int64')

In [ ]:
df['DisbursementFY'] = df['DisbursementDate'].map(lambda x: x.year)

In [ ]:
df['StateSame'] = np.where(df['State'] == df['BankState'], 1, 0)

In [ ]:
df['SBA_AppvPct'] = df['SBA_Appv'] / df['GrAppv']

In [ ]:
df['AppvDisbursed'] = np.where(df['DisbursementGross'] == df['GrAppv'], 1, 0)

In [ ]:
df.dtypes

In [ ]:
df = df.astype({'IsFranchise': 'int64', 'NewBusiness': 'int64'})

In [ ]:
df.dtypes

In [ ]:
df.drop(columns=['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'NAICS', 'ApprovalDate', 'NewExist', 'FranchiseCode',
                      'ChgOffDate', 'DisbursementDate', 'BalanceGross', 'ChgOffPrinGr', 'SBA_Appv', 'MIS_Status'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.Term.unique().sum()

In [ ]:
df['RealEstate'] = np.where(df['Term'] >= 240, 1, 0)

In [ ]:
df['GreatRecession'] = np.where(((2007 <= df['DisbursementFY']) & (df['DisbursementFY'] <= 2009)) | 
                                     ((df['DisbursementFY'] < 2007) & (df['DisbursementFY'] + (df['Term']/12) >= 2007)), 1, 0)


In [ ]:
df.DisbursementFY.unique()

In [ ]:
df = df[df.DisbursementFY <= 2010]

In [ ]:
df.shape

In [ ]:
df.describe(include = ['int', 'float', 'object'])

In [ ]:
df['DisbursedGreaterAppv'] = np.where(df['DisbursementGross'] > df['GrAppv'], 1, 0)

In [ ]:
df.DisbursedGreaterAppv.unique()

In [ ]:
df = df[df['DaysToDisbursement'] >= 0]

df.shape

In [ ]:
df.describe(include = ['int', 'float', 'object'])

In [ ]:
correlation_figure, correlation_axis = plt.subplots(figsize = (30,25))
corr_mtrx = df.corr()
correlation_axis = sns.heatmap(corr_mtrx, annot= True)

plt.xticks(rotation = 30, horizontalalignment = 'right', fontsize = 20)
plt.yticks(fontsize = 20)
plt.show()

In [ ]:
industry_group = df.groupby(['Industry'])

df_industrySum = industry_group.sum().sort_values('DisbursementGross', ascending = False)
df_industryAve = industry_group.mean().sort_values('DisbursementGross', ascending=False)

fig = plt.figure(figsize=(40,20))

ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

ax1.bar(df_industrySum.index, df_industrySum['DisbursementGross'] / 1000000000)
ax1.set_xticklabels(df_industrySum.index, rotation=30, horizontalalignment='right', fontsize=10)

ax1.set_title('Gross SBA Loan Disbursement by Industry from 1984-2010', fontsize=30)
ax1.set_xlabel('Industry', fontsize = 30)
ax1.set_ylabel('Gross Loan Disbursement (Billions)', fontsize = 30)

ax2.bar(df_industryAve.index, df_industryAve['DisbursementGross'])
ax2.set_xticklabels(df_industryAve.index, rotation=30, horizontalalignment='right', fontsize=10)

ax2.set_title('Average SBA Loan Disbursement by Industry from 1984-2010', fontsize=30)
ax2.set_xlabel('Industry',  fontsize = 30)
ax2.set_ylabel('Average Loan Disbursement',  fontsize = 30)


plt.show()

In [ ]:
fig2, ax = plt.subplots(figsize = (30,15))

ax.bar(df_industryAve.index, df_industryAve['DaysToDisbursement'].sort_values(ascending=False))
ax.set_xticklabels(df_industryAve['DaysToDisbursement'].sort_values(ascending=False).index, rotation=35,
                   horizontalalignment='right', fontsize=10)

ax.set_title('Average Days to SBA Loan Disbursement by Industry from 1984-2010', fontsize=15)
ax.set_xlabel('Industry')
ax.set_ylabel('Average Days to Disbursement')

plt.show()

In [ ]:
fig3 = plt.figure(figsize=(50, 30))

ax1a = plt.subplot(2,1,1)
ax2a = plt.subplot(2,1,2)

def stacked_setup(df, col, axes, stack_col='Default'):
    data = df.groupby([col, stack_col])[col].count().unstack(stack_col)
    data.fillna(0)

    axes.bar(data.index, data[1], label='Default')
    axes.bar(data.index, data[0], bottom=data[1], label='Paid in full')


# Number of Paid in full and defaulted loans by industry
stacked_setup(df=df, col='Industry', axes=ax1a)
ax1a.set_xticklabels(df.groupby(['Industry', 'Default'])['Industry'].count().unstack('Default').index,
                     rotation=35, horizontalalignment='right', fontsize=10)

ax1a.set_title('Number of PIF/Defaulted Loans by Industry from 1984-2010', fontsize=50)
ax1a.set_xlabel('Industry')
ax1a.set_ylabel('Number of PIF/Defaulted Loans')
ax1a.legend()

# Number of Paid in full and defaulted loans by State
stacked_setup(df=df, col='State', axes=ax2a)

ax2a.set_title('Number of PIF/Defaulted Loans by State from 1984-2010', fontsize= 50)
ax2a.set_xlabel('State')
ax2a.set_ylabel('Number of PIF/Defaulted Loans')
ax2a.legend()

plt.tight_layout()
plt.show()

In [ ]:
def_ind = df.groupby(['Industry', 'Default'])['Industry'].count().unstack('Default')
def_ind['Def_Percent'] = def_ind[1]/(def_ind[1] + def_ind[0])

def_ind

In [ ]:
def_state = df.groupby(['State', 'Default'])['State'].count().unstack('Default')
def_state['Def_Percent'] = def_state[1]/(def_state[1] + def_state[0])

def_state

In [ ]:
fig4, ax4 = plt.subplots(figsize = (30,15))

stack_data = df.groupby(['DisbursementFY', 'Default'])['DisbursementFY'].count().unstack('Default')

x = stack_data.index
y = [stack_data[1], stack_data[0]]

ax4.stackplot(x, y, labels = ['Default', 'Paid In Full'])
ax4.set_title('Number of PIF/Defaulted Loans by State from 1984-2010', fontsize = 30)

ax4.set_xlabel('Disbursement Year')
ax4.set_ylabel('Number of PIF/Defaulted Loans')
ax4.legend(loc='upper left', fontsize = 20)

plt.show()

## Model Training and Testing

Here, the plan is to one hot encode the dataframe, Normalise the dataframe by scaling it and spliting the dataset into training and testing dataframes, and train the model on the training dataset and test it on the testing and comparing the prediction and the testing target column using various metrics to find out the best possible model for the dataset.

The Classifier Models to be used are:
- Logistic Regression
- Decision Tree Classifier
- Random Forest Classifier
- Naive Bayes
- Voting Classifier


In [ ]:
df = pd.get_dummies(df)

df.head()

In [ ]:
y = df['Default']
X = df.drop('Default', axis = 1)

In [ ]:
scale = StandardScaler()
X_scld = scale.fit_transform(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scld, y, test_size=0.25)

#### Logistic Regression

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
lr = LogisticRegression(random_state = 42)

lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

print(classification_report(y_val, y_pred, digits = 3))

#### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(random_state = 42)
model_dtc = dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_val)

print(classification_report(y_val, y_pred, digits = 3))

In [ ]:
print("Testing accuracy is:",accuracy_score(y_val, y_pred))

#### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(random_state = 42)
model_rfc = rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_val)

print(classification_report(y_val, y_pred, digits = 3))

In [ ]:
print("Testing accuracy is:",accuracy_score(y_val, y_pred))

In conclusion, we get the best result from the Random Forest Classifier with an accuracy score of 94.6 on the testing dataframe.

With this project, I got to learn about the Loan Approval Process for Small Business Administration (SBA), as well about the entire process from taking the loan to disbursement of it, based on the different sectors of business. Also understood the effect of Great Recession on the Disbursement of the loans during that period, and it's effect on taking loans.